# Creación de un mapa de contaminación

##### Se pretende cargar la posición de las estaciones de medición de contaminantes junto con sus contaminantes y realizar una interpolación de estos contaminantes en todo un mallado entre las estaciones conocidas.

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
def dms2dd(input):
    "Esta función tranforma un punto expresado en coordenadas geográficas a coordenadas UTM"
    degrees,minutes,seconds,direction=re.split('[º\´"]+',str(input))
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction == 'S' or direction == 'W':
        dd *= -1
    return dd;

##### Se leeran las coordenadas de las estaciones y se filtrarán las que son de nuestro interés

In [3]:
posicion_estaciones=pd.read_excel('./data/EstacionesLocalizacion.xlsx') #Se carga el fichero

In [4]:
Estacion=[4,8,11,35,38,39,48,49,50] #Estaciones para nuestro estudio

In [5]:
posicion_estaciones=posicion_estaciones[posicion_estaciones['Estacion'].isin(Estacion)]

In [6]:
posicion_estaciones.set_index('Estacion',inplace=True)

##### Se convertirán los valores de Latitud y Longitud en coordenadas decimales

In [7]:
posicion_estaciones[['Latitud','Longitud']]=posicion_estaciones[['Latitud','Longitud']].applymap(lambda x: dms2dd(x))

In [8]:
posicion_estaciones

,Columna1,Latitud,Longitud
Estacion,,,
4,Plaza de España,40.423883,3.712197
8,Escuelas Aguirre,40.421564,3.682283
11,Av Ramon y Cajal,40.451469,3.677361
35,Plaza del carmen,40.419208,3.703167
38,Cuatro Caminos,40.445547,3.707122
39,Barrio del pilar,40.478233,3.711536
48,Paseo Castellana,40.439892,3.690372
49,Retiro,40.414444,3.682583
50,Plaza Castilla,40.465583,3.688744


In [36]:
sortLatitud=posicion_estaciones['Latitud'].sort_values()
sortLongitud=posicion_estaciones['Longitud'].sort_values()

##### Se desea obtener una matriz de puntos sobre la que realizar una interpolación. La interpolación se realizará sobre los puntos conocidos, para obtener una matriz de valores contaminantes, que representará la polución en cada espacio geografico de Madrid capital

In [97]:
Latitudes=[]
Longitudes=[]
for i in range(0,(len(sortLatitud)-1)):
    if i==7:
        lastLat=np.round(sortLatitud.iloc[i+1],decimals=6)
        lastLon=np.round(sortLongitud.iloc[i+1],decimals=6)
        Latitudes.append(lastLat)
        Longitudes.append(lastLon)
    else:
        inicioLat=sortLatitud.iloc[i]
        inicioLon=sortLongitud.iloc[i]
        finLat=sortLatitud.iloc[i+1]
        finLon=sortLongitud.iloc[i+1]
        betweenLat=np.round(np.arange(inicioLat,finLat,0.001),decimals=6)
        betweenLon=np.round(np.arange(inicioLon,finLon,0.001),decimals=6)
        Latitudes.extend(betweenLat)
        Longitudes.extend(betweenLon)

In [92]:
coordinates_map=pd.DataFrame([],index=Latitudes,columns=Longitudes)

In [104]:
#lat4=posicion_estaciones.loc['4'].round(decimals=6)
#lon4=posicion_estaciones['4','Longitud'].round(decimals=6)

In [105]:
datos_estaciones=pd.read_csv('./data/NO2_estaciones.csv')
datos_estaciones.head()

,FECHA,Estacion4,Estacion8,Estacion11,Estacion35,Estacion38,Estacion39,Estacion48,Estacion49,Estacion50
0,2015-01-01 01:00:00,83.0,168.0,93.0,109.0,102.0,104.0,108.0,136.0,127.0
1,2015-01-01 02:00:00,77.0,203.0,224.0,133.0,183.0,168.0,114.0,111.0,170.0
2,2015-01-01 03:00:00,86.0,220.0,152.0,126.0,193.0,203.0,141.0,109.0,118.0
3,2015-01-01 04:00:00,65.0,169.0,151.0,112.0,122.0,147.0,145.0,99.0,126.0
4,2015-01-01 05:00:00,53.0,154.0,106.0,109.0,97.0,159.0,142.0,103.0,140.0


In [160]:
fecha=list(datos_estaciones.FECHA)

In [173]:
ix1 = pd.MultiIndex.from_product([fecha,Latitudes])

In [174]:
mapa = pd.DataFrame(index=ix1, columns=Longitudes, dtype=np.float64)

In [175]:
#for date in fechas 
date='2015-01-01 01:00:00'
for i in Estacion:
    lat=posicion_estaciones.loc[i,'Latitud'].round(decimals=6)
    print(lat)
    lon=posicion_estaciones.loc[i,'Longitud'].round(decimals=6)
    print(lon)
    Est='Estacion'+str(i)
    sust=float(datos_estaciones[datos_estaciones['FECHA']==date][Est])
    mapa.loc[(date,lat),lon]=sust

40.423883
3.712197
40.421564
3.682283
40.451469
3.677361
40.419208
3.703167
40.445547
3.707122
40.478233
3.711536
40.439892
3.690372
40.414444
3.682583
40.465583
3.688744


In [178]:
mapa.loc[(date),:]

,3.677361,3.678361,3.679361,3.680361,3.681361,3.682283,3.682583,3.683583,3.684583,3.685583,...,3.704167,3.705167,3.706167,3.707122,3.708122,3.709122,3.710122,3.711122,3.712197,3.711536
40.414444,NaN,NaN,NaN,NaN,NaN,NaN,136.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40.415444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40.416444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40.417444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40.418444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40.419208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40.420208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40.421208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40.421564,NaN,NaN,NaN,NaN,NaN,168.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40.422564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
